In [52]:
import pandas as pd
import numpy as np
%matplotlib inline
import matplotlib.pyplot as plt
from scipy import stats
import seaborn as sns
import sqlite3
from sklearn import preprocessing
from sklearn.model_selection import GridSearchCV

In [13]:
pd.options.display.max_columns = None
players = pd.read_csv('players_mejor_3.csv')

In [14]:
players = players[players["best_of"] == 3]

In [15]:
#borramos las columnas correspondientes a variables que se encuentran desagregadas por mapa y las de resumen del partido total
players.drop(columns=['m1_kills', 'm1_assists', 'm1_deaths', 'm2_kills', 'm2_assists', 'm2_deaths', 'm3_kills', 'm3_assists', 'm3_deaths', 'kills', 'assists', 'deaths', 'hs', 'flash_assists', 'kast', 'adr', 'fkdiff', 'rating','kills_ct', 'deaths_ct', 'kddiff_ct', 'adr_ct','kast_ct','rating_ct','kills_t','deaths_t','kddiff_t','adr_t','kast_t','rating_t', 'kddiff','m1_kills_ct','m1_deaths_ct','m1_kddiff_ct','m1_adr_ct','m1_kast_ct','m1_rating_ct','m1_kills_t','m1_deaths_t','m1_kddiff_t','m1_adr_t','m1_kast_t','m1_rating_t','m2_kills_ct','m2_deaths_ct','m2_kddiff_ct','m2_adr_ct','m2_kast_ct','m2_rating_ct','m2_kills_t','m2_deaths_t','m2_kddiff_t','m2_adr_t','m2_kast_t','m2_rating_t','m3_kills_ct','m3_deaths_ct','m3_kddiff_ct','m3_adr_ct','m3_kast_ct','m3_rating_ct','m3_kills_t','m3_deaths_t','m3_kddiff_t','m3_kast_t','m3_rating_t','m3_adr_t'],axis=1,inplace=True)

In [16]:
players.sample(3)

,date,player_name,team,opponent,country,player_id,match_id,event_id,event_name,best_of,map_1,map_2,map_3,m1_hs,m1_flash_assists,m1_kast,m1_kddiff,m1_adr,m1_fkdiff,m1_rating,m2_hs,m2_flash_assists,m2_kast,m2_kddiff,m2_adr,m2_fkdiff,m2_rating,m3_hs,m3_flash_assists,m3_kast,m3_kddiff,m3_adr,m3_fkdiff,m3_rating
130955,2017-08-03,goffe,30minuter,Snabbabrillor,Sweden,10782,2313225,3039,G:loot Cup Summer 2017 Qualifier 2,3,Overpass,Mirage,NaN,16,0.0,72.2,-3,77.9,-2,1.05,7.0,2.0,53.3,-8.0,54.8,0.0,0.76,NaN,NaN,NaN,NaN,NaN,NaN,NaN
182777,2016-03-27,panda$ex,tNt,Cloud9,United States,11964,2301797,2197,DreamHack Austin 2016 Closed Qualifier,3,Cache,Cobblestone,NaN,5,NaN,70.6,-5,69.9,0,0.75,6.0,NaN,56.5,-7.0,75.3,-2.0,0.79,NaN,NaN,NaN,NaN,NaN,NaN,NaN
84694,2018-06-25,gr3eps,Wildcats,Ex-Rejected,Chile,10820,2324215,3863,HyperX Defuse 2018,3,Cache,Dust2,NaN,9,0.0,66.7,-6,82.7,-1,0.93,7.0,NaN,58.6,-7.0,58.6,-3.0,0.83,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [17]:
players.shape

(178728, 34)

In [18]:
# Crear una conexión a una base de datos en memoria
conn = sqlite3.connect(":memory:")

# Guardar los datos en una tabla temporal
players.to_sql("players", conn, if_exists="replace")

# Ejecutar la consulta SQL para filtrar los resultados
# al armar el dataset nuevo descartamos las columnas que son ids (menos match_id que lo usamos despues para transformar team y opponent), date, player_name y event_name al no utilizarlas
# tambien borramos las columnas correspondientes a valores futuros al momento de predecir por lo que no debemos mantener en entre los datos
query = """
SELECT match_id, team, opponent, country, map_1 AS first_map, m1_hs AS first_map_hs, m1_kast AS first_map_kast, m1_kddiff AS first_map_kddiff, m1_adr AS first_map_adr,
    m1_fkdiff AS first_map_fkdiff, m1_rating AS first_map_rating, map_2 AS second_map, m2_kddiff AS second_map_kddiff
FROM players;
"""

#aca lo que hacemos es para no perder los registros que solo se jugaron dos mapas porque los ganaron de corrido
#si el map3 tiene NaN no creamos el segundo registro
query2 = """
SELECT match_id, team, opponent, country, map_2 AS first_map, m2_hs AS first_map_hs, m2_kast AS first_map_kast, m2_kddiff AS first_map_kddiff, m2_adr AS first_map_adr,
    m2_fkdiff AS first_map_fkdiff, m2_rating AS first_map_rating, map_3 AS second_map, m3_kddiff AS second_map_kddiff
FROM players
WHERE map_3 IS NOT NULL;
"""

# Obtener un dataset con los registros de la primer query
players_nuevo_first = pd.read_sql_query(query, conn)
# Obterner un dataset con los registros de la segunda query
players_nuevo_second = pd.read_sql_query(query2, conn)

conn.close()

# Concatenamos ambos dataset para obtener el dataset final
players_nuevo = pd.concat([players_nuevo_first, players_nuevo_second], ignore_index=True)


In [19]:
players_nuevo.sample(3)

,match_id,team,opponent,country,first_map,first_map_hs,first_map_kast,first_map_kddiff,first_map_adr,first_map_fkdiff,first_map_rating,second_map,second_map_kddiff
177074,2298991,Refuse,Vega Squadron,Norway,Mirage,8.0,NaN,-1.0,NaN,-2.0,0.87,Cache,-2.0
211828,2322011,North,ALTERNATE aTTaX,Denmark,Cobblestone,8.0,75.0,0.0,79.6,0.0,1.16,Cache,10.0
59252,2329221,MIBR,Astralis,Brazil,Mirage,10.0,61.5,-1.0,67.5,1.0,0.86,Train,6.0


In [20]:
players_nuevo.shape

(244370, 13)

In [21]:
num_country = players_nuevo['country'].nunique()
num_first_map = players_nuevo['first_map'].nunique()
num_second_map = players_nuevo['second_map'].nunique()

# Imprimir la cantidad de valores distintos
print("Cantidad de valores distintos en la columna 'country':", num_country)
print("Cantidad de valores distintos en la columna 'first_map':", num_first_map)
print("Cantidad de valores distintos en la columna 'second_map':", num_second_map)

Cantidad de valores distintos en la columna 'country': 109
Cantidad de valores distintos en la columna 'first_map': 10
Cantidad de valores distintos en la columna 'second_map': 10


In [22]:
# hacer one hot enconder a country, first_map y second_map:
# creamos las columnas encodeadas y las formateamos
players_nuevo = pd.get_dummies(players_nuevo, columns=['country','first_map', 'second_map'])

players_nuevo.columns = players_nuevo.columns.str.lower()
BETTER_COLUMN_NAMES = {
    'country_bosnia and herzegovina': 'country_bosnia_and_herzegovina',
    'country_costa rica': 'country_costa_rica',
    'country_czech republic': 'country_czech_republic',
    'country_dominican republic': 'country_dominican_republic',
    'country_el salvador': 'country_el_salvador',
    'country_faroe islands': 'country_faroe_islands',
    'country_hong kong': 'country_hong_kong',
    'country_new zealand': 'country_new_zealand',
    'country_saudi arabia': 'country_saudi_arabia',
    'country_south africa': 'country_south_africa',
    'country_sri lanka': 'country_sri_lanka',
    'country_united arab emirates': 'country_united_arab_emirates',
    'country_united kingdom': 'country_united_kingdom',
    'country_united states': 'country_united_states'
}
players_nuevo.rename(columns=BETTER_COLUMN_NAMES, inplace=True)


In [23]:
players_nuevo.shape

(244370, 139)

In [24]:
# normalizamos todas las columnas necesarias
columnas_a_normalizar = ['first_map_hs','first_map_kast', 'first_map_kddiff', 'first_map_adr', 'first_map_fkdiff']

# Crear un objeto MinMaxScaler para normalizar las columnas
scaler = preprocessing.MinMaxScaler()

# Normalizar las columnas seleccionadas
players_nuevo[columnas_a_normalizar] = scaler.fit_transform(players_nuevo[columnas_a_normalizar])



In [25]:
# por ultimo, convertimos second_map_kddiff en las etiquetas 1 o 0 dependiendo si son positivas o negativas, respectivamente
players_nuevo['second_map_kddiff'] = np.where(players_nuevo['second_map_kddiff'] >= 0, 1, 0)

players_nuevo.sample(3)

,match_id,team,opponent,first_map_hs,first_map_kast,first_map_kddiff,first_map_adr,first_map_fkdiff,first_map_rating,second_map_kddiff,country_albania,country_algeria,country_argentina,country_armenia,country_australia,country_austria,country_azerbaijan,country_bahrain,country_bangladesh,country_belarus,country_belgium,country_bolivia,country_bosnia_and_herzegovina,country_brazil,country_brunei,country_bulgaria,country_cambodia,country_canada,country_chile,country_china,country_colombia,country_costa_rica,country_croatia,country_cyprus,country_czech_republic,country_denmark,country_dominican_republic,country_ecuador,country_egypt,country_el_salvador,country_estonia,country_faroe_islands,country_finland,country_france,country_georgia,country_germany,country_greece,country_guatemala,country_hong_kong,country_hungary,country_iceland,country_india,country_indonesia,country_iran,country_iraq,country_ireland,country_israel,country_italy,country_japan,country_jordan,country_kazakhstan,country_korea,country_kuwait,country_kyrgyzstan,country_laos,country_latvia,country_lebanon,country_libya,country_lithuania,country_luxembourg,country_macau,country_macedonia,country_malaysia,country_malta,country_mexico,country_moldova,country_mongolia,country_montenegro,country_morocco,country_myanmar,country_nepal,country_netherlands,country_new_zealand,country_norway,country_pakistan,country_palestine,country_panama,country_paraguay,country_peru,country_philippines,country_poland,country_portugal,country_qatar,country_romania,country_russia,country_saudi_arabia,country_serbia,country_singapore,country_slovakia,country_slovenia,country_south_africa,country_spain,country_sri_lanka,country_sweden,country_switzerland,country_syria,country_taiwan,country_thailand,country_tunisia,country_turkey,country_turkmenistan,country_ukraine,country_united_arab_emirates,country_united_kingdom,country_united_states,country_uruguay,country_uzbekistan,country_venezuela,country_vietnam,first_map_cache,first_map_cobblestone,first_map_default,first_map_dust2,first_map_inferno,first_map_mirage,first_map_nuke,first_map_overpass,first_map_train,first_map_vertigo,second_map_cache,second_map_cobblestone,second_map_default,second_map_dust2,second_map_inferno,second_map_mirage,second_map_nuke,second_map_overpass,second_map_train,second_map_vertigo
188497,2335511,HAVU,Gambit Youngsters,0.263158,0.675906,0.250000,0.279629,0.44,0.79,0,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False
32042,2334356,G2,NRG,0.263158,0.619403,0.400000,0.422174,0.44,1.00,0,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,F

In [26]:
# borramos los registros de players_nuevo que quedan con valores en alguna columna
indices_a_eliminar = []
for index, registro in players_nuevo.iterrows():
    # Verifica si las columnas seleccionadas contienen el valor -1
    if registro.isnull().any():
        # guardamos el indice para eliminarlo despues
        indices_a_eliminar.append(index)

# Elimina los registros en un solo paso
players_nuevo.drop(indices_a_eliminar, inplace=True)

In [27]:
#chequeamos valores nulos
players_nuevo.isnull().sum()

match_id               0
team                   0
opponent               0
first_map_hs           0
first_map_kast         0
                      ..
second_map_mirage      0
second_map_nuke        0
second_map_overpass    0
second_map_train       0
second_map_vertigo     0
Length: 139, dtype: int64

In [28]:
# guardamos el dataset que nos servira como checkpoint para hacerle luego la feature engineering
#players_nuevo.to_csv('players_preprocesado_checkpoint.csv', index=False)

#borramos luego las columnas match_id, team y opponent que no vamos a utilizar en los primeros entrenamientos
players_nuevo.drop(columns=['team', 'opponent', 'match_id'],axis=1,inplace=True)
        
players_nuevo.sample(3)

,first_map_hs,first_map_kast,first_map_kddiff,first_map_adr,first_map_fkdiff,first_map_rating,second_map_kddiff,country_albania,country_algeria,country_argentina,country_armenia,country_australia,country_austria,country_azerbaijan,country_bahrain,country_bangladesh,country_belarus,country_belgium,country_bolivia,country_bosnia_and_herzegovina,country_brazil,country_brunei,country_bulgaria,country_cambodia,country_canada,country_chile,country_china,country_colombia,country_costa_rica,country_croatia,country_cyprus,country_czech_republic,country_denmark,country_dominican_republic,country_ecuador,country_egypt,country_el_salvador,country_estonia,country_faroe_islands,country_finland,country_france,country_georgia,country_germany,country_greece,country_guatemala,country_hong_kong,country_hungary,country_iceland,country_india,country_indonesia,country_iran,country_iraq,country_ireland,country_israel,country_italy,country_japan,country_jordan,country_kazakhstan,country_korea,country_kuwait,country_kyrgyzstan,country_laos,country_latvia,country_lebanon,country_libya,country_lithuania,country_luxembourg,country_macau,country_macedonia,country_malaysia,country_malta,country_mexico,country_moldova,country_mongolia,country_montenegro,country_morocco,country_myanmar,country_nepal,country_netherlands,country_new_zealand,country_norway,country_pakistan,country_palestine,country_panama,country_paraguay,country_peru,country_philippines,country_poland,country_portugal,country_qatar,country_romania,country_russia,country_saudi_arabia,country_serbia,country_singapore,country_slovakia,country_slovenia,country_south_africa,country_spain,country_sri_lanka,country_sweden,country_switzerland,country_syria,country_taiwan,country_thailand,country_tunisia,country_turkey,country_turkmenistan,country_ukraine,country_united_arab_emirates,country_united_kingdom,country_united_states,country_uruguay,country_uzbekistan,country_venezuela,country_vietnam,first_map_cache,first_map_cobblestone,first_map_default,first_map_dust2,first_map_inferno,first_map_mirage,first_map_nuke,first_map_overpass,first_map_train,first_map_vertigo,second_map_cache,second_map_cobblestone,second_map_default,second_map_dust2,second_map_inferno,second_map_mirage,second_map_nuke,second_map_overpass,second_map_train,second_map_vertigo
46290,0.078947,0.940299,0.6,0.444020,0.48,1.53,1,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,True,False,False,False
226975,0.052632,0.514925,0.3,0.140360,0.32,0.53,1,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False,True,False,False,False,False,False,False,False,False,False
37454,0.236842,0.943497,0.7,0.678318,0.64,2.1

In [29]:
#HASTA ACA TENEMOS LISTO EL PREPROCESAMIENTO SIN FEATURE ENGINEERING

#players_nuevo.to_csv('players_preprocesado_sin_fe.csv', index=False)

players_nuevo = pd.read_csv('players_preprocesado_sin_fe.csv')
players_nuevo.sample(3)

,first_map_hs,first_map_kast,first_map_kddiff,first_map_adr,first_map_fkdiff,first_map_rating,second_map_kddiff,country_albania,country_algeria,country_argentina,country_armenia,country_australia,country_austria,country_azerbaijan,country_bahrain,country_bangladesh,country_belarus,country_belgium,country_bolivia,country_bosnia_and_herzegovina,country_brazil,country_brunei,country_bulgaria,country_cambodia,country_canada,country_chile,country_china,country_colombia,country_costa_rica,country_croatia,country_cyprus,country_czech_republic,country_denmark,country_dominican_republic,country_ecuador,country_egypt,country_el_salvador,country_estonia,country_faroe_islands,country_finland,country_france,country_georgia,country_germany,country_greece,country_guatemala,country_hong_kong,country_hungary,country_iceland,country_india,country_indonesia,country_iran,country_iraq,country_ireland,country_israel,country_italy,country_japan,country_jordan,country_kazakhstan,country_korea,country_kuwait,country_kyrgyzstan,country_laos,country_latvia,country_lebanon,country_libya,country_lithuania,country_luxembourg,country_macau,country_macedonia,country_malaysia,country_malta,country_mexico,country_moldova,country_mongolia,country_montenegro,country_morocco,country_myanmar,country_nepal,country_netherlands,country_new_zealand,country_norway,country_pakistan,country_palestine,country_panama,country_paraguay,country_peru,country_philippines,country_poland,country_portugal,country_qatar,country_romania,country_russia,country_saudi_arabia,country_serbia,country_singapore,country_slovakia,country_slovenia,country_south_africa,country_spain,country_sri_lanka,country_sweden,country_switzerland,country_syria,country_taiwan,country_thailand,country_tunisia,country_turkey,country_turkmenistan,country_ukraine,country_united_arab_emirates,country_united_kingdom,country_united_states,country_uruguay,country_uzbekistan,country_venezuela,country_vietnam,first_map_cache,first_map_cobblestone,first_map_default,first_map_dust2,first_map_inferno,first_map_mirage,first_map_nuke,first_map_overpass,first_map_train,first_map_vertigo,second_map_cache,second_map_cobblestone,second_map_default,second_map_dust2,second_map_inferno,second_map_mirage,second_map_nuke,second_map_overpass,second_map_train,second_map_vertigo
67163,0.263158,0.595949,0.433333,0.416166,0.36,1.02,0,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,True,False,False
179797,0.210526,0.684435,0.350000,0.357728,0.44,0.87,1,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,True,False,False
98087,0.184211,0.805970,0.450000,0.

In [30]:
#SI QUEREMOS SIN COUNTRIES
#prefix = 'country_'
#columns_to_drop = [col for col in players_nuevo.columns if col.startswith(prefix)]
#players_nuevo_sin_paises = players_nuevo.drop(columns=columns_to_drop)
#players_nuevo_sin_paises.to_csv('players_preprocesado_sin_paises.csv', index=False)

#players_nuevo_sin_paises = pd.read_csv('players_preprocesado_sin_paises.csv')
#players_nuevo_sin_paises.sample(3)

## Feature engineering del dataset

In [31]:
# leemos el dataset que establecimos como checkpoint antes de realizar feature engineering
players_nuevo_fe = pd.read_csv('players_preprocesado_checkpoint.csv')

In [32]:
players_nuevo_fe.sample(3)

,match_id,team,opponent,first_map_hs,first_map_kast,first_map_kddiff,first_map_adr,first_map_fkdiff,first_map_rating,second_map_kddiff,country_albania,country_algeria,country_argentina,country_armenia,country_australia,country_austria,country_azerbaijan,country_bahrain,country_bangladesh,country_belarus,country_belgium,country_bolivia,country_bosnia_and_herzegovina,country_brazil,country_brunei,country_bulgaria,country_cambodia,country_canada,country_chile,country_china,country_colombia,country_costa_rica,country_croatia,country_cyprus,country_czech_republic,country_denmark,country_dominican_republic,country_ecuador,country_egypt,country_el_salvador,country_estonia,country_faroe_islands,country_finland,country_france,country_georgia,country_germany,country_greece,country_guatemala,country_hong_kong,country_hungary,country_iceland,country_india,country_indonesia,country_iran,country_iraq,country_ireland,country_israel,country_italy,country_japan,country_jordan,country_kazakhstan,country_korea,country_kuwait,country_kyrgyzstan,country_laos,country_latvia,country_lebanon,country_libya,country_lithuania,country_luxembourg,country_macau,country_macedonia,country_malaysia,country_malta,country_mexico,country_moldova,country_mongolia,country_montenegro,country_morocco,country_myanmar,country_nepal,country_netherlands,country_new_zealand,country_norway,country_pakistan,country_palestine,country_panama,country_paraguay,country_peru,country_philippines,country_poland,country_portugal,country_qatar,country_romania,country_russia,country_saudi_arabia,country_serbia,country_singapore,country_slovakia,country_slovenia,country_south_africa,country_spain,country_sri_lanka,country_sweden,country_switzerland,country_syria,country_taiwan,country_thailand,country_tunisia,country_turkey,country_turkmenistan,country_ukraine,country_united_arab_emirates,country_united_kingdom,country_united_states,country_uruguay,country_uzbekistan,country_venezuela,country_vietnam,first_map_cache,first_map_cobblestone,first_map_default,first_map_dust2,first_map_inferno,first_map_mirage,first_map_nuke,first_map_overpass,first_map_train,first_map_vertigo,second_map_cache,second_map_cobblestone,second_map_default,second_map_dust2,second_map_inferno,second_map_mirage,second_map_nuke,second_map_overpass,second_map_train,second_map_vertigo
152610,2304422,Dignitas,Astralis,0.236842,0.605544,0.633333,0.570726,0.52,1.51,1,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False,True,False,False,False,False,False,False,False,False,False
170489,2300273,G2,CyberZen,0.157895,0.369936,0.300000,0.295467,0.52,0.59,0,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,True,False,False,False,F

In [33]:
# codificacion de variables team y opponent como el puesto que ocupaba cada equipo en el ranking de cs oficial en el dia del partido
# para esto utilizamos el dataset results.csv del mismo lugar donde descargamos el dataset que estamos utilizando
# lo que hacemos es por cada match_id, reemplazar las columnas team y opponent por el numero de ranking (columnas team_ranking y opponent_ranking)

# este dataset contiene tres registros por cada partido (el resultado de cada mapa) por lo que lo leemos y nos quedamos con cualquier de esos tres registros, ya que solo nos interesa el ranking y son los mismos valores en los tres
results = pd.read_csv('results.csv').drop_duplicates(subset='match_id', keep='first')

# vemos un ejemplo de un registro de results
results.sample(1)

,date,team_1,team_2,_map,result_1,result_2,map_winner,starting_ct,ct_1,t_2,t_1,ct_2,event_id,match_id,rank_1,rank_2,map_wins_1,map_wins_2,match_winner
785,2020-02-25,Renegades,FaZe,Inferno,12,16,2,2,5,8,7,8,4901,2339387,24,9,0,2,2


In [34]:
# creamos las columnas nuevas que van a alojar el valor de ranking
players_nuevo_fe["team_ranking"] = -1
players_nuevo_fe["opponent_ranking"] = -1

#recorremos players_nuevo_fe
for index, registro in players_nuevo_fe.iterrows():
    #buscamos el registro de results que coincida con el actual match_id del registro de players_nuevo_fe
    row = results.loc[results['match_id'] == registro['match_id']]
    #si encontramos el match en results
    if len(row) > 0:
        #asignamos las nuevas columnas del ranking de cada equipo dependiendo como venian ordenadas
        if registro['team'] == row['team_1'].values[0] and registro['opponent'] == row['team_2'].values[0]:
            players_nuevo_fe.loc[index, 'team_ranking'] = row['rank_1'].values[0]
            players_nuevo_fe.loc[index, 'opponent_ranking'] = row['rank_2'].values[0]
        elif registro['opponent'] == row['team_1'].values[0] and registro['team'] == row['team_2'].values[0]:
            players_nuevo_fe.loc[index, 'team_ranking'] = row['rank_2'].values[0]
            players_nuevo_fe.loc[index, 'opponent_ranking'] = row['rank_1'].values[0]

players_nuevo_fe.sample(3)

,match_id,team,opponent,first_map_hs,first_map_kast,first_map_kddiff,first_map_adr,first_map_fkdiff,first_map_rating,second_map_kddiff,country_albania,country_algeria,country_argentina,country_armenia,country_australia,country_austria,country_azerbaijan,country_bahrain,country_bangladesh,country_belarus,country_belgium,country_bolivia,country_bosnia_and_herzegovina,country_brazil,country_brunei,country_bulgaria,country_cambodia,country_canada,country_chile,country_china,country_colombia,country_costa_rica,country_croatia,country_cyprus,country_czech_republic,country_denmark,country_dominican_republic,country_ecuador,country_egypt,country_el_salvador,country_estonia,country_faroe_islands,country_finland,country_france,country_georgia,country_germany,country_greece,country_guatemala,country_hong_kong,country_hungary,country_iceland,country_india,country_indonesia,country_iran,country_iraq,country_ireland,country_israel,country_italy,country_japan,country_jordan,country_kazakhstan,country_korea,country_kuwait,country_kyrgyzstan,country_laos,country_latvia,country_lebanon,country_libya,country_lithuania,country_luxembourg,country_macau,country_macedonia,country_malaysia,country_malta,country_mexico,country_moldova,country_mongolia,country_montenegro,country_morocco,country_myanmar,country_nepal,country_netherlands,country_new_zealand,country_norway,country_pakistan,country_palestine,country_panama,country_paraguay,country_peru,country_philippines,country_poland,country_portugal,country_qatar,country_romania,country_russia,country_saudi_arabia,country_serbia,country_singapore,country_slovakia,country_slovenia,country_south_africa,country_spain,country_sri_lanka,country_sweden,country_switzerland,country_syria,country_taiwan,country_thailand,country_tunisia,country_turkey,country_turkmenistan,country_ukraine,country_united_arab_emirates,country_united_kingdom,country_united_states,country_uruguay,country_uzbekistan,country_venezuela,country_vietnam,first_map_cache,first_map_cobblestone,first_map_default,first_map_dust2,first_map_inferno,first_map_mirage,first_map_nuke,first_map_overpass,first_map_train,first_map_vertigo,second_map_cache,second_map_cobblestone,second_map_default,second_map_dust2,second_map_inferno,second_map_mirage,second_map_nuke,second_map_overpass,second_map_train,second_map_vertigo,team_ranking,opponent_ranking
66141,2327287,skipping school,Imperial,0.210526,0.657783,0.300000,0.330967,0.44,0.80,0,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,True,False,False,False,False,True,False,False,False,False,False,False,False,False,False,-1,-1
142730,2307063,SK,Immortals,0.289474,0.658849,0.516667,0.474604,0.56,1.32,0,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False

In [35]:
#borramos los registros de players_nuevo_fe que mantienen las columnas nuevas en -1 ya que no tenian ocurrencia en results, perdiendo datos
indices_a_eliminar = []
for index, registro in players_nuevo_fe.iterrows():
    # Verifica si las columnas seleccionadas contienen el valor -1
    if registro['team_ranking'] == -1 or registro['opponent_ranking'] == -1:
        # guardamos el indice para eliminarlo despues
        indices_a_eliminar.append(index)

# Elimina los registros en un solo paso
players_nuevo_fe.drop(indices_a_eliminar, inplace=True)

#borramos tambien las columnas match_id, team y opponent que ya no utilizamos
players_nuevo_fe.drop(columns=['team', 'opponent', 'match_id'],axis=1,inplace=True)
        
players_nuevo_fe.sample(3)

,first_map_hs,first_map_kast,first_map_kddiff,first_map_adr,first_map_fkdiff,first_map_rating,second_map_kddiff,country_albania,country_algeria,country_argentina,country_armenia,country_australia,country_austria,country_azerbaijan,country_bahrain,country_bangladesh,country_belarus,country_belgium,country_bolivia,country_bosnia_and_herzegovina,country_brazil,country_brunei,country_bulgaria,country_cambodia,country_canada,country_chile,country_china,country_colombia,country_costa_rica,country_croatia,country_cyprus,country_czech_republic,country_denmark,country_dominican_republic,country_ecuador,country_egypt,country_el_salvador,country_estonia,country_faroe_islands,country_finland,country_france,country_georgia,country_germany,country_greece,country_guatemala,country_hong_kong,country_hungary,country_iceland,country_india,country_indonesia,country_iran,country_iraq,country_ireland,country_israel,country_italy,country_japan,country_jordan,country_kazakhstan,country_korea,country_kuwait,country_kyrgyzstan,country_laos,country_latvia,country_lebanon,country_libya,country_lithuania,country_luxembourg,country_macau,country_macedonia,country_malaysia,country_malta,country_mexico,country_moldova,country_mongolia,country_montenegro,country_morocco,country_myanmar,country_nepal,country_netherlands,country_new_zealand,country_norway,country_pakistan,country_palestine,country_panama,country_paraguay,country_peru,country_philippines,country_poland,country_portugal,country_qatar,country_romania,country_russia,country_saudi_arabia,country_serbia,country_singapore,country_slovakia,country_slovenia,country_south_africa,country_spain,country_sri_lanka,country_sweden,country_switzerland,country_syria,country_taiwan,country_thailand,country_tunisia,country_turkey,country_turkmenistan,country_ukraine,country_united_arab_emirates,country_united_kingdom,country_united_states,country_uruguay,country_uzbekistan,country_venezuela,country_vietnam,first_map_cache,first_map_cobblestone,first_map_default,first_map_dust2,first_map_inferno,first_map_mirage,first_map_nuke,first_map_overpass,first_map_train,first_map_vertigo,second_map_cache,second_map_cobblestone,second_map_default,second_map_dust2,second_map_inferno,second_map_mirage,second_map_nuke,second_map_overpass,second_map_train,second_map_vertigo,team_ranking,opponent_ranking
32778,0.105263,0.675906,0.350000,0.372474,0.28,0.76,1,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,True,False,False,False,False,33,31
34413,0.342105,0.861407,0.733333,0.594211,0.40,1.75,0,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,True,False,False,

In [36]:
players_nuevo_fe.shape

(164211, 138)

In [37]:
# normalizamos todas las columnas necesarias
columnas_a_normalizar = ['team_ranking', 'opponent_ranking']

# Crear un objeto MinMaxScaler para normalizar las columnas
scaler = preprocessing.MinMaxScaler()

# Normalizar las columnas seleccionadas
players_nuevo_fe[columnas_a_normalizar] = scaler.fit_transform(players_nuevo_fe[columnas_a_normalizar])



In [38]:
players_nuevo_fe.sample(3)

,first_map_hs,first_map_kast,first_map_kddiff,first_map_adr,first_map_fkdiff,first_map_rating,second_map_kddiff,country_albania,country_algeria,country_argentina,country_armenia,country_australia,country_austria,country_azerbaijan,country_bahrain,country_bangladesh,country_belarus,country_belgium,country_bolivia,country_bosnia_and_herzegovina,country_brazil,country_brunei,country_bulgaria,country_cambodia,country_canada,country_chile,country_china,country_colombia,country_costa_rica,country_croatia,country_cyprus,country_czech_republic,country_denmark,country_dominican_republic,country_ecuador,country_egypt,country_el_salvador,country_estonia,country_faroe_islands,country_finland,country_france,country_georgia,country_germany,country_greece,country_guatemala,country_hong_kong,country_hungary,country_iceland,country_india,country_indonesia,country_iran,country_iraq,country_ireland,country_israel,country_italy,country_japan,country_jordan,country_kazakhstan,country_korea,country_kuwait,country_kyrgyzstan,country_laos,country_latvia,country_lebanon,country_libya,country_lithuania,country_luxembourg,country_macau,country_macedonia,country_malaysia,country_malta,country_mexico,country_moldova,country_mongolia,country_montenegro,country_morocco,country_myanmar,country_nepal,country_netherlands,country_new_zealand,country_norway,country_pakistan,country_palestine,country_panama,country_paraguay,country_peru,country_philippines,country_poland,country_portugal,country_qatar,country_romania,country_russia,country_saudi_arabia,country_serbia,country_singapore,country_slovakia,country_slovenia,country_south_africa,country_spain,country_sri_lanka,country_sweden,country_switzerland,country_syria,country_taiwan,country_thailand,country_tunisia,country_turkey,country_turkmenistan,country_ukraine,country_united_arab_emirates,country_united_kingdom,country_united_states,country_uruguay,country_uzbekistan,country_venezuela,country_vietnam,first_map_cache,first_map_cobblestone,first_map_default,first_map_dust2,first_map_inferno,first_map_mirage,first_map_nuke,first_map_overpass,first_map_train,first_map_vertigo,second_map_cache,second_map_cobblestone,second_map_default,second_map_dust2,second_map_inferno,second_map_mirage,second_map_nuke,second_map_overpass,second_map_train,second_map_vertigo,team_ranking,opponent_ranking
71742,0.315789,0.750533,0.450000,0.361005,0.48,1.02,1,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False,0.193717,0.096859
93909,0.210526,0.881663,0.566667,0.406335,0.56,1.58,0,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False

In [39]:
players_nuevo_fe.shape

(164211, 138)

In [47]:
#HASTA ACA TENEMOS LISTO EL PREPROCESAMIENTO CON FEATURE ENGINEERING

#players_nuevo_fe.to_csv('players_preprocesado_con_fe.csv', index=False)

players_nuevo_fe = pd.read_csv('players_preprocesado_con_fe.csv')

players_nuevo_fe.sample(3)

,first_map_hs,first_map_kast,first_map_kddiff,first_map_adr,first_map_fkdiff,first_map_rating,second_map_kddiff,country_albania,country_algeria,country_argentina,country_armenia,country_australia,country_austria,country_azerbaijan,country_bahrain,country_bangladesh,country_belarus,country_belgium,country_bolivia,country_bosnia_and_herzegovina,country_brazil,country_brunei,country_bulgaria,country_cambodia,country_canada,country_chile,country_china,country_colombia,country_costa_rica,country_croatia,country_cyprus,country_czech_republic,country_denmark,country_dominican_republic,country_ecuador,country_egypt,country_el_salvador,country_estonia,country_faroe_islands,country_finland,country_france,country_georgia,country_germany,country_greece,country_guatemala,country_hong_kong,country_hungary,country_iceland,country_india,country_indonesia,country_iran,country_iraq,country_ireland,country_israel,country_italy,country_japan,country_jordan,country_kazakhstan,country_korea,country_kuwait,country_kyrgyzstan,country_laos,country_latvia,country_lebanon,country_libya,country_lithuania,country_luxembourg,country_macau,country_macedonia,country_malaysia,country_malta,country_mexico,country_moldova,country_mongolia,country_montenegro,country_morocco,country_myanmar,country_nepal,country_netherlands,country_new_zealand,country_norway,country_pakistan,country_palestine,country_panama,country_paraguay,country_peru,country_philippines,country_poland,country_portugal,country_qatar,country_romania,country_russia,country_saudi_arabia,country_serbia,country_singapore,country_slovakia,country_slovenia,country_south_africa,country_spain,country_sri_lanka,country_sweden,country_switzerland,country_syria,country_taiwan,country_thailand,country_tunisia,country_turkey,country_turkmenistan,country_ukraine,country_united_arab_emirates,country_united_kingdom,country_united_states,country_uruguay,country_uzbekistan,country_venezuela,country_vietnam,first_map_cache,first_map_cobblestone,first_map_default,first_map_dust2,first_map_inferno,first_map_mirage,first_map_nuke,first_map_overpass,first_map_train,first_map_vertigo,second_map_cache,second_map_cobblestone,second_map_default,second_map_dust2,second_map_inferno,second_map_mirage,second_map_nuke,second_map_overpass,second_map_train,second_map_vertigo,team_ranking,opponent_ranking
121662,0.263158,0.703625,0.450000,0.414528,0.56,1.08,1,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,0.065445,0.028796
76586,0.263158,0.688699,0.416667,0.487712,0.44,1.08,0,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False

## Definición de métrica:

Elegimos ACCURACY (precisión).
La precisión es la proporción de predicciones correctas realizadas por el modelo. Se calcula dividiendo el número de predicciones correctas entre el número total de predicciones.
Elegimos esta métrica ya que es especialmente util en clasificación y además es fácil de interpretar.
Su desventaja principal se encuentra en problemas cuyas clases en el conjunto de datos están desbalanceadas, cosa que no presentará inconvenientes en nuestro dataset (valor obtenido en tp1 -> kddiff positivo = 50,07% / kddiff negativo = 49,03%)

## División del dataset:

### Sin feature engineering

In [48]:
from sklearn.model_selection import train_test_split

# Dividir el conjunto de datos en variable de entrada (entradas) y variable objetivo (salida)

# Dividir los datos en conjunto de entrenamiento y conjunto de prueba
# 60% train, 20% test, 20% validation
train, not_train = train_test_split(players_nuevo, test_size=0.4, random_state=42)
validation, test = train_test_split(not_train, test_size=0.5, random_state=42)

train_y = train['second_map_kddiff']
train_x = train.drop(columns=['second_map_kddiff'])

validation_y = validation['second_map_kddiff']
validation_x = validation.drop(columns=['second_map_kddiff'])

test_y = test['second_map_kddiff']
test_x = test.drop(columns=['second_map_kddiff'])

# Mostramos las dimensiones finales de cada porcion del dataset
train_x.shape, train_y.shape, validation_x.shape, validation_y.shape, test_x.shape, test_y.shape




((140025, 135), (140025,), (46675, 135), (46675,), (46676, 135), (46676,))

### Con feature engineering

In [56]:
from sklearn.model_selection import train_test_split

# Dividir el conjunto de datos en variable de entrada (entradas) y variable objetivo (salida)

# Dividir los datos en conjunto de entrenamiento y conjunto de prueba
# 60% train, 20% test, 20% validation
train_fe, not_train_fe = train_test_split(players_nuevo_fe, test_size=0.4, random_state=42)
validation_fe, test_fe = train_test_split(not_train_fe, test_size=0.5, random_state=42)

train_y_fe = train_fe['second_map_kddiff']
train_x_fe = train_fe.drop(columns=['second_map_kddiff'])

validation_y_fe = validation_fe['second_map_kddiff']
validation_x_fe = validation_fe.drop(columns=['second_map_kddiff'])

test_y_fe = test_fe['second_map_kddiff']
test_x_fe = test_fe.drop(columns=['second_map_kddiff'])

# Mostramos las dimensiones finales de cada porcion del dataset
train_x_fe.shape, train_y_fe.shape, validation_x_fe.shape, validation_y_fe.shape, test_x_fe.shape, test_y_fe.shape




((98526, 137), (98526,), (32842, 137), (32842,), (32843, 137), (32843,))

## Entrenamiento de modelos:

### Regresion logística

In [50]:
from sklearn.linear_model import LogisticRegression
from sklearn import metrics

# sin feature engineering
regression_model = LogisticRegression(random_state=42, max_iter=1000)
regression_model.fit(train_x, train_y)
y_pred = regression_model.predict(test_x)
print("Accuracy on test: " + str(metrics.accuracy_score(test_y, y_pred)))

# con feature engineering
regression_model_fe = LogisticRegression(random_state=42, max_iter=1000)
regression_model_fe.fit(train_x_fe, train_y_fe)
y_pred_fe = regression_model_fe.predict(test_x_fe)
print("Accuracy en test (con FE): " + str(metrics.accuracy_score(test_y_fe, y_pred_fe)))

Accuracy on test: 0.5775987659610935
Accuracy en test (con FE): 0.5876137989830406


### Random forest

In [45]:
from sklearn.ensemble import RandomForestClassifier

# sin feature engineering
forest_model = RandomForestClassifier()
forest_model.fit(train_x, train_y)
y_pred = forest_model.predict(test_x)
print("Accuracy on test: " + str(metrics.accuracy_score(test_y, y_pred)))

# sin feature engineering
forest_model_fe = RandomForestClassifier()
forest_model_fe.fit(train_x_fe, train_y_fe)
y_pred_fe = forest_model_fe.predict(test_x_fe)
print("Accuracy on test (con FE): " + str(metrics.accuracy_score(test_y_fe, y_pred_fe)))

Accuracy on test: 0.5602022452652327
Accuracy on test (con FE): 0.5741558322930305


### K nearest neighbors (KNN)

In [46]:
from sklearn.neighbors import KNeighborsClassifier

#Parametros varios:
K = 5

# sin feature engineering
kkn_model = KNeighborsClassifier(n_neighbors=K)
kkn_model.fit(train_x, train_y)
y_pred = kkn_model.predict(test_x)
print("Accuracy on test: " + str(metrics.accuracy_score(test_y, y_pred)))

# con feature engineering
kkn_model_fe = KNeighborsClassifier(n_neighbors=K)
kkn_model_fe.fit(train_x_fe, train_y_fe)
y_pred_fe = kkn_model_fe.predict(test_x_fe)
print("Accuracy on test (con FE): " + str(metrics.accuracy_score(test_y_fe, y_pred_fe)))

Accuracy on test: 0.5372568343474162
Accuracy on test (con FE): 0.5466309411442316


### Dummie Classifier

In [114]:
from sklearn.dummy import DummyClassifier

# sin feature engineering
dummy_clf = DummyClassifier(strategy='uniform')
dummy_clf.fit(train_x, train_y)
y_pred = dummy_clf.predict(test_x)
print("Accuracy on test: " + str(metrics.accuracy_score(test_y, y_pred)))

# con feature engineering
dummy_clf_fe = DummyClassifier(strategy='uniform')
dummy_clf_fe.fit(train_x_fe, train_y_fe)
y_pred_fe = dummy_clf_fe.predict(test_x_fe)
print("Accuracy on test (con FE): " + str(metrics.accuracy_score(test_y_fe, y_pred_fe)))

Accuracy on test: 0.49717199417259406
Accuracy on test (con FE): 0.5004110464939256


## Definición de hiperparametros implementando GridSearch:

### Regresion logística

In [61]:
params = {
    'penalty': ['l2'],
    'C': [0.1, 1.0, 10.0]
}

lr_grid_search = GridSearchCV(regression_model_fe, params, cv=5)
lr_grid_search.fit(train_x_fe, train_y_fe)
y_pred_fe = lr_grid_search.predict(validation_x_fe)

print("Accuracy en validation: " + str(metrics.accuracy_score(validation_y_fe, y_pred_fe)))

print("Mejores parámetros encontrados:")
print(lr_grid_search.best_params_)
print("Puntuación media en el conjunto de validación:")
print(lr_grid_search.best_score_)

Accuracy en validation: 0.5823640460386091
Mejores parámetros encontrados:
{'C': 10.0, 'penalty': 'l2'}
Puntuación media en el conjunto de validación:
0.5856728365228179


### Random forest

In [66]:
params = {
    'n_estimators': [50, 100, 200], 
    'max_depth': [5, 10, 15], 
    'max_features': ['sqrt', 'log2', 2, 5]
}

rf_grid_search = GridSearchCV(forest_model_fe, params, cv=5)
rf_grid_search.fit(train_x_fe, train_y_fe)
y_pred_fe = forest_model_fe.predict(validation_x_fe)

print("Accuracy en validation: " + str(metrics.accuracy_score(validation_y_fe, y_pred_fe)))

print("Mejores parámetros encontrados:")
print(rf_grid_search.best_params_)
print("Puntuación media en el conjunto de validación:")
print(rf_grid_search.best_score_)

Accuracy en validation: 0.5715547165215273
Mejores parámetros encontrados:
{'max_depth': 15, 'max_features': 'sqrt', 'n_estimators': 100}
Puntuación media en el conjunto de validación:
0.5814100033754244


### K nearest neighbors (KNN)

In [67]:
params = {'n_neighbors': [3, 5, 7],
          'weights': ['uniform', 'distance'],
          'p': [1, 2]}

knn_grid_search = GridSearchCV(kkn_model_fe, params, cv=5)
knn_grid_search.fit(train_x_fe, train_y_fe)
y_pred_fe = knn_grid_search.predict(validation_x_fe)

print("Accuracy en validation: " + str(metrics.accuracy_score(validation_y_fe, y_pred_fe)))

print("Mejores parámetros encontrados:")
print(knn_grid_search.best_params_)
print("Puntuación media en el conjunto de validación:")
print(knn_grid_search.best_score_)

Accuracy en validation: 0.5566043480908592
Mejores parámetros encontrados:
{'n_neighbors': 7, 'p': 1, 'weights': 'distance'}
Puntuación media en el conjunto de validación:
0.5525850952930946


## Reducción de dimensionalidad
### Principal Component Analysis (PCA)

In [ ]:
from sklearn.decomposition import PCA

pca = PCA(n_components=15)
pca.fit(train_x_fe)

print("Importancias de las nuevas dimensiones:", pca.explained_variance_ratio_)  
nX = pca.transform(X)

Se aplio feature egineer a partir de las columnas team y opponet se crearon dos features nuevas team_ranking y opponent_ranking obteniendo nueva informacion medible del ranking que tenian los equipos el dia del match.

No aplicamos Quantile transformation ya que nuestro dataset tiene una distribucion normal. Tampoco se trata de crear nuevas features de datos pasados ya que no tenemos datos de tiempo y podria introducir datos erroreos.

Conclusiones:
    como podemos ver en los test realizados anteriormente no encontramos una mejora significativa luego de aplicar tecnicas de feature engineer. 